<a href="https://colab.research.google.com/github/Barinderkaur15/SellBy_ETL/blob/master/challenge_dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#for Postgrate driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
# Start a SparkSession
import findspark
findspark.init()
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("challenege").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

--2020-02-04 05:54:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.69MB/s    in 0.2s    

2020-02-04 05:54:48 (4.69 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



# **Extract**

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://*****-bucket.s3.amazonaws.com/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True)

In [0]:
#To check our data frame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [4]:
#to count number of rows
df.count()

1767753

# **Transform**

In [5]:
#droping NAN vlaues 
df_clean_jwelery1=df.dropna()
df_clean_jwelery1.count()

1767394

In [6]:
#Deleting dupicate Rows 
df_clean_jwelery2=df_clean_jwelery1.distinct()
df_clean_jwelery2.count()

1767394

In [7]:
#checking data type before transformation 
df_clean_jwelery2.dtypes


[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
#Changing the cloumn format
#importing the function 
from pyspark.sql.types import IntegerType,DateType
from pyspark.sql.functions import col

df_clean_jwelery3 = df_clean_jwelery2.withColumn("review_date",df_clean_jwelery2["review_date"].cast(DateType()))
df_clean_jwelery4 = df_clean_jwelery3.withColumn("customer_id",df_clean_jwelery3["customer_id"].cast(IntegerType()))
df_clean_jwelery5 = df_clean_jwelery4.withColumn("product_parent",df_clean_jwelery4["product_parent"].cast(IntegerType()))
df_clean_jwelery6 = df_clean_jwelery5.withColumn("star_rating",df_clean_jwelery5["star_rating"].cast(IntegerType()))
df_clean_jwelery7= df_clean_jwelery6.withColumn("helpful_votes",df_clean_jwelery6["helpful_votes"].cast(IntegerType()))
clean_jwelery_df=df_clean_jwelery7.withColumn("total_votes",df_clean_jwelery7["total_votes"].cast(IntegerType()))
clean_jwelery_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25310224|R1005USL5JYZRA|B00BG5WB78|     442189212|Lady Gold Tone Me...|         Jewelry|          4|            4|          4|   N|                Y|     Inexpensive fun|I ordered this fo...| 2013-09-04|
|         US|   19996332|R1007OQ3NNV3RD|B0051IX9RW|     893505108|Bling Jewelry Bla...|         Jewelry|          1|    

In [9]:
clean_jwelery_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [10]:
#creating a dataframe for Postgres
#Table 1 : Review id 
review_id_df=clean_jwelery_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
#review_id check 1 
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [11]:
#checking Table 1
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005USL5JYZRA|   25310224|B00BG5WB78|     442189212| 2013-09-04|
|R1007OQ3NNV3RD|   19996332|B0051IX9RW|     893505108| 2013-06-13|
| R100M7CG8G15W|    2689743|B00GL7PRQI|     994203804| 2015-01-04|
|R102N945VZXRYA|    7801778|B00IXXYJR6|       9545313| 2014-09-15|
|R1036HQCDVDYZE|   36088460|B002SCEG26|     257100538| 2010-02-08|
| R103AUP7TMOLY|   45589994|B00M3E6BZY|     496113502| 2015-03-22|
|R103DIHV9RV1XC|   14202882|B00A5U08AC|     870816223| 2014-07-12|
|R103G95MZXEIGD|   43396325|B005NRJ8U8|     520530881| 2014-10-08|
|R103GNQ7U01KHG|   23698386|B004ZMBIW8|     724759429| 2012-12-18|
|R103SQMC26YT0J|   31699158|B00EEJEO5A|     755341131| 2014-09-08|
|R103XBEYQCG6ST|    2536953|B00J6TAG4G|     663679725| 2014-11-19|
|R104A86O5NY51A|   18479519|B0047ENX8G|     208039327| 2014-02

In [0]:
#creating product table using distinct values only 
from pyspark.sql.functions import col, countDistinct
distinct_value=clean_jwelery_df.distinct()

product_df=distinct_value.select(["product_id","product_title"])
#droping duplicate value 
product_jwelery_df= product_df.dropDuplicates()

#product_df.select(countDistinct("product_id")).show()

In [13]:
#checking distinct 
product_jwelery_df.count()


574016

In [14]:
#To check unique value 
product_df.select(countDistinct("product_id")).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    574016|
+--------------------------+



In [15]:
#creating Customer_df
from pyspark.sql.functions import col
customers=clean_jwelery_df.groupby("customer_id").count()
customers_df = customers.select(col("customer_id").alias("customer_id"), col("count").alias("customer_count"))
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16885399|             4|
|   16407298|             1|
|   21196854|             1|
|   24142937|             1|
|   43731249|             1|
|   41658829|             1|
|   12411613|             1|
|   44105454|             1|
|     514807|             1|
|   32501131|             1|
|   11567869|             2|
|   13803822|             1|
|   37452471|             9|
|   17314604|             2|
|   33955796|             2|
|   35194823|             1|
|   26431178|             4|
|   31618415|             2|
|   36932022|             2|
|   15383445|             7|
+-----------+--------------+
only showing top 20 rows



In [16]:
#creating Vine table
vine_df=clean_jwelery_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005USL5JYZRA|          4|            4|          4|   N|
|R1007OQ3NNV3RD|          1|            1|          1|   N|
| R100M7CG8G15W|          4|            0|          0|   N|
|R102N945VZXRYA|          5|            0|          0|   N|
|R1036HQCDVDYZE|          5|            1|          1|   N|
| R103AUP7TMOLY|          4|            0|          0|   N|
|R103DIHV9RV1XC|          5|            0|          0|   N|
|R103G95MZXEIGD|          3|            0|          0|   N|
|R103GNQ7U01KHG|          1|            0|          1|   N|
|R103SQMC26YT0J|          5|            0|          0|   N|
|R103XBEYQCG6ST|          4|            0|          0|   N|
|R104A86O5NY51A|          5|            2|          2|   N|
|R104XC8EVVPJ9M|          5|            0|          0|   N|
|R104Y8UZVF0RS9|          5|            

# **Load**

In [17]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
# Configure settings for RDS
#Hiding the Password 
mode = "append"
jdbc_url="jdbc:postgresql://challengeetl.cx9tn8vhsc47.us-east-2.rds.amazonaws.com:5432/challenge"
config = {"user":"postgres",
          "password": "*****",
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrame to review table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_jewel1_table', mode=mode, properties=config)

In [0]:
# Write DataFrame to products table in RDS
product_jwelery_df.write.jdbc(url=jdbc_url, table='products_jewel1', mode=mode, properties=config)

In [0]:
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_jewel1', mode=mode, properties=config)

In [0]:
# Write DataFrame to vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table_jewel1', mode=mode, properties=config)